<a href="https://colab.research.google.com/github/rayssabanti/Python-Analise-de-dados-Portfolio/blob/main/An%C3%A1lise_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpeza e Exploração de Dados

## Importando as bibliotecas

In [56]:
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot') 
from matplotlib.pyplot import figure
import math
import re
import datetime

%matplotlib inline 
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.set_option('display.max.rows', None)

## Importando a base de dados

In [57]:
dataset = pd.read_csv('dados\movies.csv')

## Limpeza dos dados

#### Buscando as colunas com nulos e preenchendo com 0 (caso existam)

In [59]:
# Finding the missing values
for col in dataset.columns:
  faltante = dataset[col].isnull().sum()
  print (f'{col} - {faltante} ')
  if faltante != 0:
    dataset[col] = dataset[col].fillna(0)
    print (f'{col} - alterada ')
  else:
    print (f'{col} - ok ')

name - 0 
name - ok 
rating - 77 
rating - alterada 
genre - 0 
genre - ok 
year - 0 
year - ok 
released - 2 
released - alterada 
score - 3 
score - alterada 
votes - 3 
votes - alterada 
director - 0 
director - ok 
writer - 3 
writer - alterada 
star - 1 
star - alterada 
country - 3 
country - alterada 
budget - 2171 
budget - alterada 
gross - 189 
gross - alterada 
company - 17 
company - alterada 
runtime - 4 
runtime - alterada 


#### Alterando os tipos das colunas

In [60]:
dataset['budget'] = dataset['budget'].astype('int64')
dataset['gross'] = dataset['gross'].astype('int64')
dataset['runtime'] = dataset['runtime'].astype('int64')
dataset['votes'] = dataset['runtime'].astype('int64')

In [61]:
dataset.dtypes

name         object
rating       object
genre        object
year          int64
released     object
score       float64
votes         int64
director     object
writer       object
star         object
country      object
budget        int64
gross         int64
company      object
runtime       int64
dtype: object

#### Correção nas colunas RELEASED E YEAR

Existem casos onde a coluna Year tem anos diferentes da Released, foi identificado que a Year pode conter o ano de liberação do teaser e que a Released seria o gabarito. 

*   Extração do ano da coluna Released
*   Existem alguns casos na released que o ano não está preenchido, necessário correção. 
*   Comparação com a Year



In [62]:
dataset['ano']  =  [re.findall('[0-9]{4,}', str(x)) for x in dataset['released']]
dataset['ano']  =  [re.sub('[^A-Za-z0-9]+', '', str(x)) for x in dataset['ano']]

In [63]:
for linha, linha2, posicao in zip(dataset['ano'], dataset['year'], range(dataset['year'].shape[0])):
      if linha == '':
        print(f'Na posição: {posicao} o valor da coluna released é vazio, alterando para {linha2}')
        dataset.loc[posicao, 'ano'] = linha2

dataset['ano'] = dataset['ano'].astype('int64')

for linha, linha2, posicao in zip(dataset['ano'], dataset['year'], range(dataset['year'].shape[0])):
      if linha2 != linha:
        print(f'Na posição: {posicao} o valor da coluna year {linha2} é diferente do released, alterando para {linha}')
        dataset.loc[posicao, 'year'] = linha
        
del dataset['ano']

Na posição: 5728 o valor da coluna released é vazio, alterando para 2010
Na posição: 5730 o valor da coluna released é vazio, alterando para 2010
Na posição: 8 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 11 o valor da coluna year 1980 é diferente do released, alterando para 1984
Na posição: 21 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 33 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 40 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 49 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 54 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 59 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 62 o valor da coluna year 1980 é diferente do released, alterando para 1981
Na posição: 79 o valor da coluna year 1980 é diferente do relea

Extraindo os meses do released

In [64]:
dataset['month'] = [re.findall('^\w+', str(x)) for x in dataset['released']]
dataset['month']  =  [re.sub('[^A-Za-z0-9]+', '', str(x)) for x in dataset['month']]
dataset['month']

0            June
1            July
2            June
3            July
4            July
5             May
6            June
7        December
8            June
9             May
10       December
11        October
12       December
13      September
14           July
15        October
16            May
17       December
18       February
19       December
20       February
21          April
22         August
23         August
24           June
25       December
26          March
27         August
28           June
29       February
30      September
31           July
32         August
33          April
34       December
35        October
36            May
37        October
38       December
39          March
40        October
41           July
42           June
43          April
44        October
45           July
46           June
47       November
48            May
49            May
50      September
51      September
52           June
53           June
54          April
55       N

#### Extraindo os meses (em números também)

In [65]:

for mes, posicao in zip(dataset['month'],range(dataset['month'].shape[0])):
    meses = ['January','February','March', 'April','May', 'June', 'July', 'August', 'September','October',  'November', 'December']
    if mes not in meses:
        dataset.loc[posicao, 'month'] = 0
        print('mudado')
        

mudado
mudado
mudado
mudado
mudado
mudado
mudado
mudado
mudado
mudado
mudado
mudado


In [66]:
 dataset['month'].drop_duplicates()

0           June
1           July
5            May
7       December
11       October
13     September
18      February
21         April
22        August
26         March
47      November
74       January
201            0
Name: month, dtype: object

Transformando o nome dos meses em números - em uma nova coluna

Primeira forma:

In [67]:
dataset['month_number'] = ""
dataset.loc[dataset['month'] == 'January', 'month_number'] = 1
dataset.loc[dataset['month'] == 'February', 'month_number'] = 2
dataset.loc[dataset['month'] == 'March', 'month_number'] = 3
dataset.loc[dataset['month'] == 'April', 'month_number'] = 4
dataset.loc[dataset['month'] == 'May', 'month_number'] = 5
dataset.loc[dataset['month'] == 'June', 'month_number'] = 6
dataset.loc[dataset['month'] == 'July', 'month_number'] = 7
dataset.loc[dataset['month'] == 'August', 'month_number'] = 8
dataset.loc[dataset['month'] == 'September', 'month_number'] = 9
dataset.loc[dataset['month'] == 'October', 'month_number'] = 10
dataset.loc[dataset['month'] == 'November', 'month_number'] = 11
dataset.loc[dataset['month'] == 'December', 'month_number'] = 12



Segunda forma:

In [68]:
dataset.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,month,month_number
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,146,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772,Warner Bros.,146,June,6
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,104,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106,Columbia Pictures,104,July,7
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,124,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067,Lucasfilm,124,June,6
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,88,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539,Paramount Pictures,88,July,7
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,98,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344,Orion Pictures,98,July,7
